In [ ]:
# label_dict={'14783134 15697333 14854817 14925479':0,'14847385 14844587 14848641 14847398':1, \
#      '14786237 15697082 14722731 14924977':2, '14844093 15705739 14854331 15699885':3, \
#      '15632285 15706536 14721977 14925219':4, '15630486 15702410 14718849 15709093':5, \
#      '14924216 14781104 14717848 14791612':6, '14782903 15634620 15638402 15706300':7, \
#     '14858934 15636660 15704193 14849963':8, '15709098 14716590 14924703 14779559':9, \
#      '14726332 14728344 14854542 14844591':10,'14856354 14844592':11, \
#      '15710359 14847407 14845602 14859696':12, '14794687 14782344':13, \
#      '14925756 15639967 14853254 14728639':14, '14844593 14924945':15, \
#     '14844856 14724258 14925237 14854807':16, '14852788 14717848 15639958 15632020':17, \
#      '14784131 14858934 14784131 14845064':18}

# import pandas as pd
# import pickle
# import gc

# print('loading datasets......')
# train_data = pd.read_csv('./data/train.csv')
# new_label = []
# for i in range(len(train_data)):
#      new_label.append(label_dict[train_data.loc[i]["label"]])

# train_data["new_label"] = new_label
# train_data.to_csv("./data/new_train_data.csv", index=False)

In [ ]:
import pandas as pd
import pickle
import gc
# handle raw data

print('loading datasets......')
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

print('{} lines in train datasets'.format(len(train_data)))
print('{} lines in test datasets'.format(len(test_data)))

all_text = []

for row in range(len(train_data)):
    all_text.append(train_data.loc[row]["name"].split())
    all_text.append(train_data.loc[row]["description"].split())

for row in range(len(test_data)):
    all_text.append(test_data.loc[row]["name"].split())
    all_text.append(test_data.loc[row]["description"].split())

with open("./data/raw_text.pkl", "wb") as f:
    pickle.dump(all_text, f)

In [ ]:
# Word2Vec
from gensim.models.word2vec import Word2Vec

embed_size = 100
window = 4
min_count = 1

model = Word2Vec(all_text, vector_size=embed_size, window=window, min_count=min_count, workers=4)
model.wv.save_word2vec_format("./data/wordvec_embsize_{}_window_{}_mincount_{}.bin".format(embed_size, window, min_count), binary=True)

In [ ]:
model.wv.vectors

In [ ]:
from gensim.models.word2vec import Word2Vec, KeyedVectors
embedding_model = KeyedVectors.load_word2vec_format("./data/wordvec_embsize_100_window_4_mincount_3.bin", binary=True).key_to_index

import pandas as pd

train_data = pd.read_csv("./data/new_train_data.csv")
new_train = pd.DataFrame(columns=["name", "description", "new_label"])
for i in range(len(train_data)):
    flag = 0
    for name in train_data.iloc[i]["name"].split():
        if name in embedding_model:
            flag += 1
            break
    
    for desc in train_data.iloc[i]["description"].split():
        if desc in embedding_model:
            flag += 1
            break
    if flag == 2:
        print("sadas")
        new_train.append(train_data.iloc[i][["name", "description", "new_label"]], ignore_index = True)
new_train.to_csv("./data/new_new_data.csv", index=False)